<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled289.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Math 2 / Math 5 governance maze simulation (Colab-ready)
# Saves MP4 + NPZ (+ optional PNG frames). Walls drawn as perpendicular matchsticks.

import os
import random
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

try:
    import imageio.v2 as imageio
except Exception:
    import imageio

# =========================
# CONFIG
# =========================
N = 21
NUM_PLAYERS = 4
STEPS = 400
BAR_ADD_EVERY = 2          # add barrier every k steps (Math 2)
PRUNE_EVERY = 10         # set >0 for Math 5 (e.g. 10)
PRUNE_K = 3

BREAK_PROB = 0.08
BREAK_COST = 2
INIT_CREDITS = 8

SHOW_PATHS = True
SAVE_MP4 = True
SAVE_NPZ = True
SAVE_FRAMES = False

FPS = 6
DPI = 140
SEED = 1

OUTDIR = "math5_outputs"
MP4_NAME = os.path.join(OUTDIR, "math5_sim.mp4")
NPZ_NAME = os.path.join(OUTDIR, "math5_run.npz")
FRAMES_DIR = os.path.join(OUTDIR, "frames")

random.seed(SEED)
np.random.seed(SEED)

os.makedirs(OUTDIR, exist_ok=True)
if SAVE_FRAMES:
    os.makedirs(FRAMES_DIR, exist_ok=True)

# =========================
# HELPERS
# =========================
def rc2i(r, c): return r * N + c
def i2rc(i): return divmod(i, N)

def neighbors(i):
    r, c = i2rc(i)
    out = []
    if r > 0: out.append(rc2i(r-1, c))
    if c < N-1: out.append(rc2i(r, c+1))
    if r < N-1: out.append(rc2i(r+1, c))
    if c > 0: out.append(rc2i(r, c-1))
    return out

def manhattan(a, b):
    ra, ca = i2rc(a)
    rb, cb = i2rc(b)
    return abs(ra-rb) + abs(ca-cb)

def edge(a, b):
    return (a, b) if a < b else (b, a)

# =========================
# RINGS + TRACKS
# =========================
ring = np.zeros((N, N), dtype=int)
for r in range(N):
    for c in range(N):
        ring[r, c] = min(r, c, N-1-r, N-1-c)

MAX_RING = int(ring.max())
CENTER = rc2i(N//2, N//2)

def build_ring_track(k):
    r1, c1 = k, k
    r2, c2 = N-1-k, N-1-k
    if r1 > r2 or c1 > c2:
        return []
    pts = []
    for c in range(c1, c2+1): pts.append(rc2i(r1, c))
    for r in range(r1+1, r2+1): pts.append(rc2i(r, c2))
    if r2 > r1:
        for c in range(c2-1, c1-1, -1): pts.append(rc2i(r2, c))
    if c2 > c1:
        for r in range(r2-1, r1, -1): pts.append(rc2i(r, c1))
    return pts

RING_TRACKS = [build_ring_track(k) for k in range(MAX_RING+1)]

# =========================
# BARRIERS (MATCHSTICKS)
# =========================
blocked = set()

def is_blocked(a, b):
    return edge(a, b) in blocked

def can_move(a, b):
    return b in neighbors(a) and not is_blocked(a, b)

def add_random_barrier():
    for _ in range(600):
        a = random.randrange(N*N)
        r, c = i2rc(a)
        if random.random() > (ring[r, c]+1)/(MAX_RING+1):
            continue
        neigh = neighbors(a)
        if not neigh:
            continue
        b = random.choice(neigh)
        e = edge(a, b)
        if e not in blocked:
            blocked.add(e)
            return

def prune_barriers(k):
    if not blocked:
        return
    for _ in range(min(k, len(blocked))):
        blocked.remove(random.choice(tuple(blocked)))

# =========================
# PLAYERS
# =========================
corners = [(0,0),(0,N-1),(N-1,N-1),(N-1,0)]
players = []
for p in range(NUM_PLAYERS):
    r0, c0 = corners[p]
    pos = rc2i(r0, c0)
    tr = RING_TRACKS[0]
    ix = min(range(len(tr)), key=lambda i: abs(tr[i]-pos))
    players.append({
        "pos": pos,
        "ring": 0,
        "track_ix": ix,
        "laps": 0,
        "credits": INIT_CREDITS,
        "reached": False
    })

def try_step(a, b, pl):
    if can_move(a, b):
        return True
    if random.random() < BREAK_PROB and pl["credits"] >= BREAK_COST:
        pl["credits"] -= BREAK_COST
        return True
    return False

def attempt_inward(pl):
    if pl["laps"] <= 0 or pl["ring"] >= MAX_RING:
        return
    r, c = i2rc(pl["pos"])
    if ring[r, c] != pl["ring"]:
        return
    cand = []
    for nb in neighbors(pl["pos"]):
        rr, cc = i2rc(nb)
        if ring[rr, cc] == pl["ring"] + 1:
            cand.append(nb)
    if not cand:
        return
    cand.sort(key=lambda x: manhattan(x, CENTER))
    tgt = cand[0]
    if try_step(pl["pos"], tgt, pl):
        pl["pos"] = tgt
        pl["ring"] += 1
        tr = RING_TRACKS[pl["ring"]]
        pl["track_ix"] = min(range(len(tr)), key=lambda i: abs(tr[i]-pl["pos"]))
        pl["laps"] = 0

def move_player(pl):
    if pl["reached"]:
        return True
    if pl["pos"] == CENTER:
        pl["reached"] = True
        return True

    tr = RING_TRACKS[pl["ring"]]
    nxt_ix = (pl["track_ix"] + 1) % len(tr)
    if nxt_ix == 0:
        pl["laps"] += 1
    nxt = tr[nxt_ix]

    if nxt not in neighbors(pl["pos"]) or try_step(pl["pos"], nxt, pl):
        pl["pos"] = nxt
        pl["track_ix"] = nxt_ix
        attempt_inward(pl)
        if pl["pos"] == CENTER:
            pl["reached"] = True
        return True

    return False

# =========================
# GRAPH + METRICS
# =========================
def build_graph():
    G = nx.Graph()
    G.add_nodes_from(range(N*N))
    for a in range(N*N):
        for b in neighbors(a):
            if a < b and not is_blocked(a, b):
                G.add_edge(a, b)
    return G

def compute_metrics():
    G = build_graph()
    reachable = []
    path_lens = []
    for pl in players:
        if pl["pos"] == CENTER:
            reachable.append(True)
            path_lens.append(0)
        else:
            try:
                d = nx.shortest_path_length(G, pl["pos"], CENTER)
                reachable.append(True)
                path_lens.append(int(d))
            except:
                reachable.append(False)
                path_lens.append(-1)
    try:
        comp = nx.node_connected_component(G, CENTER)
        comp_size = len(comp)
    except:
        comp_size = 0
    return {
        "reachable": np.array(reachable),
        "path_len": np.array(path_lens),
        "center_component_size": int(comp_size),
        "num_barriers": int(len(blocked)),
        "total_credits": int(sum(pl["credits"] for pl in players)),
        "num_reached": int(sum(1 for pl in players if pl["reached"]))
    }

# =========================
# RENDER FRAME (PERPENDICULAR WALLS)
# =========================
def render_frame(t, stuck):
    fig, ax = plt.subplots(figsize=(7,7), dpi=DPI)
    ax.imshow(ring, origin="lower")

    # Draw blocked edges as short perpendicular walls (matchsticks)
    L = 0.45
    for (a, b) in blocked:
        ra, ca = i2rc(a)
        rb, cb = i2rc(b)
        mx = (ca + cb) / 2.0
        my = (ra + rb) / 2.0
        dr = rb - ra
        dc = cb - ca
        if dc != 0:
            ax.plot([mx, mx], [my - L, my + L], linewidth=4, color="black")
        else:
            ax.plot([mx - L, mx + L], [my, my], linewidth=4, color="black")

    cr, cc = i2rc(CENTER)
    ax.plot(cc, cr, marker="o", markersize=10, color="white", markeredgecolor="black")
    ax.text(cc+0.3, cr, "CENTER", weight="bold", color="white")

    for i, pl in enumerate(players, start=1):
        r, c = i2rc(pl["pos"])
        ax.plot(c, r, 'o')
        ax.text(c+0.2, r+0.2, f"P{i}({pl['credits']})", weight="bold")

    if SHOW_PATHS:
        G = build_graph()
        for pl in players:
            if pl["pos"] == CENTER:
                continue
            try:
                path = nx.shortest_path(G, pl["pos"], CENTER)
                rr = [i2rc(x)[0] for x in path]
                cc2 = [i2rc(x)[1] for x in path]
                ax.plot(cc2, rr, linewidth=1.2, alpha=0.9)
            except:
                pass

    ax.set_xticks([]); ax.set_yticks([])
    ax.set_title(f"t={t}  barriers={len(blocked)}  stuck={stuck}  pruneEvery={PRUNE_EVERY}")

    fig.canvas.draw()
    buf = np.asarray(fig.canvas.buffer_rgba())
    img = buf[:, :, :3].copy()
    plt.close(fig)
    return img

# =========================
# RUN + SAVE
# =========================
hist_blocked = []
hist_players = []
hist_metrics = []

writer = None
if SAVE_MP4:
    try:
        writer = imageio.get_writer(MP4_NAME, fps=FPS)
    except Exception as e:
        SAVE_MP4 = False
        print("MP4 writer unavailable. In Colab try:")
        print("!pip -q install imageio imageio-ffmpeg")
        print("Error:", e)

stuck = 0
for t in range(1, STEPS+1):

    if BAR_ADD_EVERY and t % BAR_ADD_EVERY == 0:
        add_random_barrier()

    if PRUNE_EVERY and t % PRUNE_EVERY == 0:
        prune_barriers(PRUNE_K)

    for pl in players:
        if not move_player(pl):
            stuck += 1

    if SAVE_NPZ:
        hist_blocked.append(list(blocked))
        hist_players.append([dict(pl) for pl in players])
        hist_metrics.append(compute_metrics())

    img = render_frame(t, stuck)

    if SAVE_FRAMES:
        imageio.imwrite(os.path.join(FRAMES_DIR, f"frame_{t:04d}.png"), img)

    if SAVE_MP4 and writer is not None:
        writer.append_data(img)

    if all(pl["reached"] for pl in players):
        break

if writer is not None:
    writer.close()

if SAVE_NPZ:
    np.savez_compressed(
        NPZ_NAME,
        ring=ring,
        blocked=np.array(hist_blocked, dtype=object),
        players=np.array(hist_players, dtype=object),
        metrics=np.array(hist_metrics, dtype=object),
        params=dict(
            N=N, NUM_PLAYERS=NUM_PLAYERS, STEPS=STEPS,
            BAR_ADD_EVERY=BAR_ADD_EVERY, PRUNE_EVERY=PRUNE_EVERY, PRUNE_K=PRUNE_K,
            BREAK_PROB=BREAK_PROB, BREAK_COST=BREAK_COST, INIT_CREDITS=INIT_CREDITS,
            SHOW_PATHS=SHOW_PATHS, SEED=SEED, FPS=FPS, DPI=DPI
        )
    )

print("Done. Saved outputs to:", OUTDIR)
print("MP4:", MP4_NAME if SAVE_MP4 else "(disabled)")
print("NPZ:", NPZ_NAME if SAVE_NPZ else "(disabled)")
print("Frames:", FRAMES_DIR if SAVE_FRAMES else "(disabled)")

In [ ]:
import os
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import animation
from IPython.display import clear_output

# ---------------------------------------
# Settings
# ---------------------------------------
N = 21                       # grid size (odd)
NUM_PLAYERS = 4              # 4 corners
MAX_STEPS = 300              # steps to simulate

BAR_ADD_EVERY = 2            # add barriers every k steps
INNER_BIAS = 0.65            # bias barrier placements toward inner rings

INIT_CREDITS = 8

# Player flash when a rule is broken
player_flash = {}   # pid -> frames left
FLASH_TTL = 2        # number of frames to flash

BREAK_PROB = 0.08            # chance to break a blocked move
BREAK_COST = 2               # cost per break

PRUNE_EVERY = 0              # 0 disables pruning
PRUNE_K = 3                  # number of barriers removed per prune event

SHOW_PATHS = True            # overlay shortest paths to center (expensive)
SAVE_FRAMES = False          # save frames as PNG
SAVE_MP4 = True          # save as mp4
MP4_FPS = 20
DPI = 120

OUTDIR = "governance_game_out"
os.makedirs(OUTDIR, exist_ok=True)

# ---------------------------------------
# Helpers
# ---------------------------------------
def rc2i(r, c):
    return r * N + c

def i2rc(i):
    return (i // N, i % N)

CENTER = rc2i(N // 2, N // 2)

def manhattan(a, b):
    ra, ca = i2rc(a)
    rb, cb = i2rc(b)
    return abs(ra - rb) + abs(ca - cb)

# ---------------------------------------
# Ring tracks (outer rim inward)
# ---------------------------------------
def ring_index(i):
    r, c = i2rc(i)
    return min(r, c, N - 1 - r, N - 1 - c)

def build_ring_tracks():
    tracks = []
    max_ring = (N - 1) // 2
    for k in range(max_ring + 1):
        cells = []
        top = k
        bot = N - 1 - k
        left = k
        right = N - 1 - k

        # single cell at center ring
        if top == bot and left == right:
            tracks.append([rc2i(top, left)])
            continue

        # top row (left->right)
        for c in range(left, right + 1):
            cells.append(rc2i(top, c))
        # right col (top+1->bot-1)
        for r in range(top + 1, bot):
            cells.append(rc2i(r, right))
        # bottom row (right->left)
        for c in range(right, left - 1, -1):
            cells.append(rc2i(bot, c))
        # left col (bot-1->top+1)
        for r in range(bot - 1, top, -1):
            cells.append(rc2i(r, left))

        tracks.append(cells)
    return tracks

RING_TRACKS = build_ring_tracks()

# ---------------------------------------
# Barriers ("matchsticks") as blocked edges
# ---------------------------------------
blocked = set()  # store undirected edges as (min(a,b), max(a,b))

def edge(a, b):
    return (a, b) if a < b else (b, a)

def neighbors(i):
    r, c = i2rc(i)
    out = []
    if r > 0: out.append(rc2i(r - 1, c))
    if r < N - 1: out.append(rc2i(r + 1, c))
    if c > 0: out.append(rc2i(r, c - 1))
    if c < N - 1: out.append(rc2i(r, c + 1))
    return out

def can_move(a, b):
    return edge(a, b) not in blocked

def add_random_barrier():
    # bias toward inner rings: pick a random cell with probability proportional to (ring+1)^p
    weights = []
    cells = list(range(N * N))
    for i in cells:
        k = ring_index(i)
        w = (k + 1) ** 2
        weights.append(w)

    # sample a cell
    chosen = random.choices(cells, weights=weights, k=1)[0]

    # choose a neighbor
    neigh = neighbors(chosen)
    if not neigh:
        return
    b = random.choice(neigh)

    # bias toward inner region by rejecting some outer placements
    k = ring_index(chosen)
    max_ring = (N - 1) // 2
    inner_frac = k / max_ring if max_ring > 0 else 1.0
    if random.random() > (INNER_BIAS * inner_frac + (1 - INNER_BIAS) * 0.5):
        return

    blocked.add(edge(chosen, b))

def prune_barriers(k=PRUNE_K):
    if len(blocked) == 0:
        return
    rem = random.sample(list(blocked), k=min(k, len(blocked)))
    for e in rem:
        blocked.discard(e)

# ---------------------------------------
# Build graph for shortest paths / components
# ---------------------------------------
def build_graph():
    G = nx.Graph()
    for i in range(N * N):
        G.add_node(i)
    for i in range(N * N):
        for j in neighbors(i):
            if can_move(i, j):
                G.add_edge(i, j)
    return G

# ---------------------------------------
# Initialize players
# ---------------------------------------
corners = [(0,0),(0,N-1),(N-1,N-1),(N-1,0)]
players = []
for p in range(NUM_PLAYERS):
    r0, c0 = corners[p]
    pos = rc2i(r0, c0)
    track = RING_TRACKS[0]
    ix = min(range(len(track)), key=lambda i: abs(track[i]-pos))

    players.append({
        "id": p,
        "pos": pos,
        "ring": 0,
        "track_ix": ix,
        "laps": 0,
        "credits": INIT_CREDITS,
        "reached": False
    })

def try_step(a, b, pl):
    if can_move(a, b):
        return True

    if random.random() < BREAK_PROB and pl["credits"] >= BREAK_COST:
        pl["credits"] -= BREAK_COST
        player_flash[pl["id"]] = FLASH_TTL
        return True

    return False

# ---------------------------------------
# Movement rule: follow ring until lap complete, then attempt inward
# ---------------------------------------
def advance_player(pl):
    if pl["reached"]:
        return

    if pl["pos"] == CENTER:
        pl["reached"] = True
        return

    ring = pl["ring"]
    track = RING_TRACKS[ring]

    # If in inner ring center, just move to center if possible
    if ring == (N - 1) // 2:
        return

    # Step along current ring track
    next_ix = (pl["track_ix"] + 1) % len(track)
    a = track[pl["track_ix"]]
    b = track[next_ix]

    moved = False

    # Try ring move
    if try_step(a, b, pl):
        pl["track_ix"] = next_ix
        pl["pos"] = b
        moved = True

        # completed a lap?
        if pl["track_ix"] == 0:
            pl["laps"] += 1

    # If lap completed at least once, attempt inward move preferentially
    if pl["laps"] >= 1 and not pl["reached"]:
        # Candidate inward neighbors (ring index increases by 1)
        inward = []
        for nb in neighbors(pl["pos"]):
            if ring_index(nb) == ring + 1:
                inward.append(nb)

        # choose inward move that reduces distance to center most
        if inward:
            inward_sorted = sorted(inward, key=lambda x: manhattan(x, CENTER))
            target = inward_sorted[0]
            if try_step(pl["pos"], target, pl):
                pl["pos"] = target
                pl["ring"] = ring + 1

                # Update track index to nearest on new ring track
                new_track = RING_TRACKS[pl["ring"]]
                pl["track_ix"] = min(range(len(new_track)),
                                     key=lambda i: abs(new_track[i] - pl["pos"]))
                pl["laps"] = 0

# ---------------------------------------
# Metrics
# ---------------------------------------
def metrics():
    G = build_graph()
    comp_size = 0
    if CENTER in G:
        comp = nx.node_connected_component(G, CENTER)
        comp_size = len(comp)

    reached = sum(1 for pl in players if pl["pos"] == CENTER)
    stuck = sum(1 for pl in players if not pl["reached"] and len([nb for nb in neighbors(pl["pos"]) if can_move(pl["pos"], nb)]) == 0)

    # shortest path lengths
    spl = []
    for pl in players:
        try:
            spl.append(nx.shortest_path_length(G, pl["pos"], CENTER))
        except Exception:
            spl.append(np.inf)

    return {
        "num_barriers": len(blocked),
        "center_component_size": comp_size,
        "num_reached": reached,
        "stuck": stuck,
        "avg_spl": float(np.mean([x for x in spl if np.isfinite(x)]) if any(np.isfinite(x) for x in spl) else np.inf)
    }

# ---------------------------------------
# Render
# ---------------------------------------
def render_frame(t, stuck_count):
    fig, ax = plt.subplots(figsize=(7, 7), dpi=DPI)

    ax.imshow(np.zeros((N, N)), origin="lower", cmap="Greys", vmin=0, vmax=1)

    # barriers as thick segments between cell centers
    for (a, b) in blocked:
        ra, ca = i2rc(a)
        rb, cb = i2rc(b)
        ax.plot([ca, cb], [ra, rb], linewidth=3, color="black")

    # center marker
    cr, cc = i2rc(CENTER)
    ax.plot(cc, cr, marker="o", markersize=10, color="white", markeredgecolor="black")
    ax.text(cc + 0.3, cr, "CENTER", fontsize=10, weight="bold", color="white")

    # players (flash on rule-break)
    for pl in players:
        r, c = i2rc(pl["pos"])
        pid = pl.get("id", None)

        if pid is not None and player_flash.get(pid, 0) > 0:
            ax.add_patch(
                plt.Rectangle((c - 0.5, r - 0.5), 1, 1,
                              color="red", alpha=0.15, linewidth=0)
            )
            ax.scatter([c], [r], s=900, marker="o",
                       color="red", alpha=0.20, linewidths=0)
            ax.scatter([c], [r], s=220, marker="o",
                       color="red", edgecolors="black", linewidths=1.5)
        else:
            ax.scatter([c], [r], s=120, marker="o",
                       edgecolors="black", linewidths=1.2)

        label = f"P{(pl.get('id', 0) + 1)}({pl['credits']})"
        ax.text(c + 0.2, r + 0.2, label, fontsize=10, weight="bold")

    # Decay flash counters
    for pid in list(player_flash.keys()):
        player_flash[pid] -= 1
        if player_flash[pid] <= 0:
            del player_flash[pid]

    # optional shortest paths overlay
    if SHOW_PATHS:
        G = build_graph()
        for pl in players:
            if pl["pos"] == CENTER:
                continue
            try:
                path = nx.shortest_path(G, pl["pos"], CENTER)
                pts = [i2rc(x)[::-1] for x in path]  # (c,r)
                xs = [p[0] for p in pts]
                ys = [p[1] for p in pts]
                ax.plot(xs, ys, linewidth=1.5, alpha=0.5)
            except Exception:
                pass

    ax.set_title(f"t={t} | barriers={len(blocked)} | stuck={stuck_count}")
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout()
    plt.close(fig)

    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    img = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape(h, w, 4)[:, :, :3]
    return img

# ---------------------------------------
# Run
# ---------------------------------------
hist_blocked = []
hist_players = []
hist_metrics = []

frames = []

for t in range(MAX_STEPS):

    # add barriers periodically
    if (t % BAR_ADD_EVERY) == 0:
        add_random_barrier()

    # prune barriers periodically (if enabled)
    if PRUNE_EVERY and (t % PRUNE_EVERY) == 0 and t > 0:
        prune_barriers(PRUNE_K)

    # move players
    for pl in players:
        advance_player(pl)

    m = metrics()
    hist_metrics.append(m)
    hist_blocked.append(list(blocked))
    hist_players.append([{k: v for k, v in pl.items()} for pl in players])

    # render frame
    img = render_frame(t, m["stuck"])
    frames.append(img)

# Save outputs
np.savez_compressed(
    os.path.join(OUTDIR, "governance_game_history.npz"),
    hist_metrics=np.array(hist_metrics, dtype=object),
    hist_blocked=np.array(hist_blocked, dtype=object),
    hist_players=np.array(hist_players, dtype=object)
)

# Save MP4 if requested
if SAVE_MP4:
    import imageio.v2 as imageio
    mp4_path = os.path.join(OUTDIR, "governance_game.mp4")
    imageio.mimwrite(mp4_path, frames, fps=MP4_FPS)

print("Done. Outputs in:", OUTDIR)

Done. Outputs in: governance_game_out


In [ ]:
from google.colab import drive
drive.mount('/content/drive')